## Langevin Dynamics
This part of the tutorial covers how to write molecular dynamics data to a dump file in python and then visualize the output with OVITO.

A spherical particle of size $R$ and mass $m$, dispersed in a solvent of viscosity $\eta$, will experience a friction $\gamma$ and a stochastic force $\zeta(t)$. If $v(t)$ denotes the velocity of the particle, its motion will be described by the Langevin equation:

$m \frac{dv(t)}{dt} = -\gamma v(t) + \zeta(t)$, 

The foolowing code provide a simple molecular dynamics solver that simulates the motion of non-interacting particles in the canonical ensemble using a Langevin thermostat:


### Step 1: Import library
The following the head library we need to this code:

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import dump #This is pre-written to build up a dump file suit for OVITO to visualise the data - see Step 7

# Define global physical constants
Avogadro = 6.02214086e23
Boltzmann = 1.38064852e-23

### Step 2: This function enforces reflective boundary conditions. All particles that hit a wall  have their velocity updated in opposite direction.

Notes:
-pos: atomic positions (ndarray)
-vels: atomic velocity (ndarray, updated if collisions detected)
-box: simulation box size (tuple)


In [ ]:
def wallHitCheck(pos, vels, box):
    ndims = len(box)
    for i in range(ndims):
        vels[((pos[:,i] <= box[i][0]) | (pos[:,i] >= box[i][1])),i] *= -1

### Step 3: A simple forward Euler integrator that moves the system in time 

In [ ]:
def integrate(pos, vels, forces, mass,  dt):
    pos += vels * dt
    vels += forces * dt / mass[np.newaxis].T

### Step 4: Computes the Langevin force for all particles 

Notes:
-mass: particle mass (ndarray)
-temp: temperature (float)
-relax: thermostat constant (float)
-dt: simulation timestep (float)

-> returns forces (ndarray)

#### EXERCISE: complete the function below, needed for computing force in Langevin:

In [ ]:
def computeForce(mass, vels, temp, relax, dt):

    natoms, ndims = vels.shape

    # Your code here...
        # sigma = ...
        # noise = ...
        # force = ...
        
    return force

#### SOLUTION (execute this cell if you wish to see the solution)

In [ ]:
# %load solutions/computeForce.py

### Step 5: This is the main function that solves Langevin's equations for a system of natoms usinga forward Euler scheme, and returns an output list that stores the time and the temperture.
 
Notes:

-natoms (int): number of particles
-temp (float): temperature (in Kelvin)
-mass (float): particle mass (in Kg)
-relax (float): relaxation constant (in 1/seconds)
-dt (float): simulation timestep (s)
-nsteps (int): total number of steps the solver performs
-box (tuple): simulation box size (in meters) of size dimensions x 2
    e.g. box = ((-1e-9, 1e-9), (-1e-9, 1e-9)) defines a 2D square
-ofname (string): filename to write output to
-freq (int): write output every 'freq' steps
-[radius]: particle radius (for visualization)
    
-> Returns a list (of size nsteps x 2) containing the time and temperature.

In [ ]:

def run(**args):

    natoms, box, dt, temp = args['natoms'], args['box'], args['dt'], args['temp']
    mass, relax, nsteps   = args['mass'], args['relax'], args['steps']
    ofname, freq, radius = args['ofname'], args['freq'], args['radius']
    
    dim = len(box)
    pos = np.random.rand(natoms,dim)

    for i in range(dim):
        pos[:,i] = box[i][0] + (box[i][1] -  box[i][0]) * pos[:,i]

    vels = np.random.rand(natoms,dim)
    mass = np.ones(natoms) * mass / Avogadro
    radius = np.ones(natoms) * radius
    step = 0

    output = [] #initialization 

    while step <= nsteps:
        step += 1
        forces = computeForce(mass, vels, temp, relax, dt) # Compute all forces
        integrate(pos, vels, forces, mass, dt) # Move the system in time
        wallHitCheck(pos,vels,box) # Check if any particle has collided with the wall
        ins_temp = np.sum(np.dot(mass, (vels - vels.mean(axis=0))**2)) / (Boltzmann * dim * natoms)
        output.append([step * dt, ins_temp])  # Compute output (temperature)
        
        if not step%freq:
            dump.writeOutput(ofname, natoms, step, box, radius=radius, pos=pos, v=vels)

    return np.array(output)


### Step 6: Run and plot the data, time $vs$ temperture

In [ ]:

if __name__ == '__main__':

    params = {
        'natoms': 1000,
        'temp': 300,
        'mass': 0.001,
        'radius': 120e-12,
        'relax': 1e-13,
        'dt': 1e-15,
        'steps': 10000,
        'freq': 100,
        'box': ((0, 1e-8), (0, 1e-8), (0, 1e-8)),
        'ofname': 'traj-hydrogen-3D.dump'
        }

    output = run(**params)

    plt.plot(output[:,0] * 1e12, output[:,1])
    plt.xlabel('Time (ps)')
    plt.ylabel('Temp (K)')
    plt.show()

### Step 7: Create a dump file availiable for OVITO - load to see the code

In [ ]:
# %load dump.py